# How to backtest
This tutorial will show you an overview of how to use the framework to do backtest.  It assumes that you are already familiar with the ```data``` module. The primary module that will be used in this tutorial is the ```engine``` module.

## The normal structure of a backtest project
Usually, a backtest project should have these sections:
- The data pipeline
- The strategy definition
- The backtest configuration
- The analysis of the results

Each section could be in the same file for a small project, in different files for a bigger project and maybe also in different directories for even larger projects.

In the get_started section of the official documentation, there is a an example.  We will walk through the example to see where each parts are found.
```python
from backtest import Strategy, Backtest
from backtest.indicators import IndicatorSet, TA
from backtest.data import FetchCharts, ToTSData, Cache, PadNan
import backtest.engine.functional as F
from datetime import datetime

# -------------------------------------------------------------------------------------------------------------- #
# Section A
class MyStrategy(Strategy):
    def run(self, data, timestep):
        for ticker in data.main.tickers:
            chart = data.main[ticker].chart
            if len(chart) > 2 and F.crossover(chart["MACD"], chart["MACD_SIGNAL"]) and chart["MACD"].iloc[-1] < 0:
                if ticker not in self.broker.portfolio.long:
                    self.broker.buy_long(ticker, 500)
            if ticker in self.broker.portfolio.long and F.crossunder(chart["MACD"], chart["MACD_SIGNAL"]):
                self.broker.sell_long(ticker, 500)
# End of section A
# -------------------------------------------------------------------------------------------------------------- #


# -------------------------------------------------------------------------------------------------------------- #
# Section B
TICKERS = ["META", "AMZN", "AAPL", "NVDA", "GOOGL", "MSFT", "TSLA"]
data_pipeline = FetchCharts(TICKERS, auto_adjust=False) | PadNan() | ToTSData() | Cache()
# End of section B
# -------------------------------------------------------------------------------------------------------------- #

# -------------------------------------------------------------------------------------------------------------- #
# Section C
bt = Backtest(data_pipeline.get(datetime(2010, 1, 1), datetime(2020, 1, 1)),
              strategy=MyStrategy(),
              indicators=IndicatorSet(TA.MACD()))
results = bt.run()
# End of section C
# -------------------------------------------------------------------------------------------------------------- #

# -------------------------------------------------------------------------------------------------------------- #
# Section D
print(results)
# End of section D
# -------------------------------------------------------------------------------------------------------------- #
```

In the example the section:
- A) Defines the strategy
- B) Defines the data pipeline
- C) Configures and runs the backtest
- D) Analyse the results

As you can see, all of these sections are fairly simple in that example, but can become extensivly complex if needed.  The syntax is really flexible.

## Strategy
In this section, you will see how to create a strategy that fits your need.  You will see how to interact with the bank account, the broker and the portfolio.

### 1. Derive the class
The first step is to derive a class from the ```Strategy``` class.  The only required method to implement is the ```run``` method.  This is where the logic of the strategy is implemented.  It can be simple if-else clauses, it can be some decision trees or complex machine learning algorithms.  You can also override the \_\_init\_\_ method to add some dynamic attributes to your class.  You can add any method you want and attributes you want with two exceptions: ```init``` and ```__call__```.  You should not override those methods.  Notice that it is written ```init``` not ```__init__```.

When a strategy is release using the serve module its state needs to be saved to disk.  To do so, there is a default save and load method implemented that store the state as a pickle file.  If you want to store the state in another format, you can override those methods.

### 2. Data Interaction
The run method is called at every step and passes the current data to the strategy via the data parameter.  This parameter contains a ```RecordsBucket``` object.  This object contains few attributes defined below.  This object contains all the records(charts) of all assets across all time resolutions (if multiple were used).  To access a given time resolution, you can use the ```[]``` operator with the desired ```timedelta``` object or its index in the ```available_time_res``` attribute.  This will return a ```Records``` object.  The ```main``` attribute is an alias to access the ```Records``` object associated with the main time resolution.

The ```Records``` is sort of a set of individual ```Record``` for a single time resolution.  Each record contains the price chart of the asset and all the added features, but also some other information such as the rate of dividends (quarterly, annually, etc).  The data is kind of structured as a tree:
```
RecordsBucket
├ Records
┊ ┕ Record
┊ ┊  ┕ pd.DataFrame (The chart)
```

### 3. Interactions with the Account
It can be useful to know how much available cash remains in the account to calculate how much money to invest in a trade.  To do so, you can interact with the account that is referenced as the ```account``` attribute of the Strategy.  It is directly connected to the backtest engine.  It is strongly recommonded to not modify the state of the account for example depositing money or adding collateral, even though nothing blocks you from modifying doing because it could cause bugs and give wrong backtest results.  What you can do with the account is access the ```available_cash``` property, the ```collateral``` property or the ```transactions``` property, but make sure you do not modify it.

### 4. Interactions with the portfolio
It can be useful to know what positions are currently open to perform actions.  To do so, yoi can access the ```porfolio``` attribute of the Strategy.  Again, you must not modify its state.  You can access the ```long``` or the ```short``` attributes to access the open positions.  Again, make sure to not modify anything.  

### 5. Interaction with the broker
Your strategy will interact with the broker to open/close positions.  You can use these methods:
- ```buy_long```: to open a long position
- ```sell_long```: to close a long position
-  ```sell_short```: to open a short position
-  ```buy_short```: to close a short position.

You must not change the state of the broker other than from these methods, again to avoid bugs and false results.  If you want to know what are the pending transactions, you can access the the ```pending_orders``` property of the broker.

### Example
In the following example, we will implement a Strategy that buys a security when the 7 day simple moving average start increasing and the price is over the 50 days moving simple moving average.  It sells if the 7 day moving average start decreasing.  It will use only a single